# 處理Tongue_coating_classification 資料

## **1. 數據處理部分**

### **1.1 資料夾結構初始化**

```python
os.makedirs(os.path.join(images_dir, "train"), exist_ok=True)
os.makedirs(os.path.join(images_dir, "val"), exist_ok=True)
os.makedirs(os.path.join(labels_dir, "train"), exist_ok=True)
os.makedirs(os.path.join(labels_dir, "val"), exist_ok=True)
```

- 建立 `images` 和 `labels` 資料夾，並在各自底下創建 `train` 與 `val` 子資料夾。
- YOLO 格式的數據需要分為訓練集（`train`）和驗證集（`val`），圖片及標註文件應按照該結構進行存放。

---

### **1.2 標籤定義**

```python
class_labels = {
    "black tongue coating": 0,
    "map tongue coating": 1,
    "purple tongue coating": 2,
    # ...
}
```

- 使用字典（`dict`）對應每個舌苔類別與其整數 ID。
- 例如：`"black tongue coating": 0` 代表「黑色舌苔」，其類別編號為 0。

---

### **1.3 標註文件生成**

```python
def create_txt_from_polygon(points, img_width, img_height, class_id):
    # 將多邊形頂點轉為相對座標（0～1），並依照 YOLO 格式輸出
    # YOLO Segmentation 格式： class_id x1 y1 x2 y2 ...
    ...
```

- 讀取每個物件（多邊形）的點座標，並轉換為相對圖片寬高的比例（介於 0 到 1 之間）。
- 將結果以 **`class_id x1 y1 x2 y2 ... xn yn`** 的形式寫入 `.txt` 檔。

---

### **1.4 數據處理流程**

1. **迭代每種類別資料夾**  
   ```python
   for class_name, class_id in class_labels.items():
       class_dir = os.path.join(data_dir, class_name)
       # 驗證該資料夾是否存在並進行後續處理
   ```
   - 逐一讀取各類別的資料夾。

2. **處理每個 JSON 標註文件**  
   ```python
   with open(json_path, 'r') as f:
       data = json.load(f)
       # 取得 data['shapes'] 或其他必要欄位
   ```
   - 讀取與當前類別對應的 JSON 檔案，解析其中的多邊形資訊（`shapes`）。

3. **生成標註文件**  
   ```python
   txt_output_path = os.path.join(labels_dir, subset, filename_no_ext + ".txt")
   # 根據圖片檔名與對應的資料集 (train/val)，將標註寫入 .txt
   ```
   - 呼叫先前的 `create_txt_from_polygon` 函數，將標註以 YOLO 格式寫入對應的 `.txt` 檔。

4. **分配圖片到訓練集和驗證集**  
   ```python
   # 例如: if random.random() < 0.8 => train, else => val
   shutil.copy(img_path, os.path.join(images_dir, subset, filename))
   ```
   - 隨機抽樣（如 80% 放進 `train`，20% 放進 `val`），將圖片複製到對應目錄下。

---

### **1.5 建立 `data.yaml`**

```python
data_yaml = {
    "train": "/path/to/dataset/images/train",
    "val": "/path/to/dataset/images/val",
    "nc": len(class_labels),
    "names": list(class_labels.keys())
}
```

- 生成 YOLO 的配置檔 `data.yaml`：
  - `train` 與 `val` 需指向對應的圖片目錄。
  - `nc` 是類別數目。
  - `names` 是類別名稱清單，順序需與標籤定義對應。

---

## **2. 模型訓練部分**

### **2.1 載入 YOLO 模型**

```python
from ultralytics import YOLO
```

- 使用 `ultralytics` 套件中的 YOLO 類進行訓練。
- 以下示範以 YOLOv8 的分割模型（`yolov8s-seg`）為主。

---

### **2.2 訓練 YOLO 模型**

```bash
!yolo task=segment mode=train model=yolov8s-seg.pt data=data.yaml epochs=50 imgsz=640
```

- `task=segment`：指定任務為分割（Segmentation）。
- `model=yolov8s-seg.pt`：使用 YOLOv8 預訓練分割模型。
- `data=data.yaml`：指定前面建立的 `data.yaml` 檔。
- `epochs=50`：訓練週期數（可依需求調整）。
- `imgsz=640`：輸入圖片縮放大小（640×640）。

---

## **總結**

1. **數據準備**  
   - 整理原始數據（包含圖片與標註）至符合 YOLO Segmentation 所需的資料夾結構與格式。  
   - 生成 `data.yaml` 作為 YOLO 的配置檔。

2. **模型訓練**  
   - 使用 `ultralytics` 套件中的 YOLOv8 預訓練分割模型，進行微調（fine-tuning）。
   - 根據任務需求設定 `epochs`、`imgsz` 等參數。

當數據集與配置都準備妥當後，即可使用指令或程式碼來執行訓練。

---

## **YOLOv8 Segmentation 數據格式與要求**

### **1. 資料夾結構**

```plaintext
dataset/
├── images/
│   ├── train/
│   │   ├── image1.jpg
│   │   ├── image2.jpg
│   │   └── ...
│   ├── val/
│   │   ├── image3.jpg
│   │   ├── image4.jpg
│   │   └── ...
├── labels/
│   ├── train/
│   │   ├── image1.txt
│   │   ├── image2.txt
│   │   └── ...
│   ├── val/
│   │   ├── image3.txt
│   │   ├── image4.txt
│   │   └── ...
└── data.yaml
```

- `images/` 與 `labels/` 皆分為 `train`、`val` 兩個子資料夾。
- `data.yaml` 為訓練配置檔，記錄路徑與類別資訊。

---

### **2. 圖片格式**

- 支援 `.jpg`、`.png` 等常見影像格式。
- 圖片檔名和標註檔名需對應（副檔名除外）。

---

### **3. 標註文件格式**

YOLOv8 的 Segmentation 任務中，每個標註檔（`.txt`）的每一行描述一個物件，多邊形格式為：

```plaintext
class_id x1 y1 x2 y2 x3 y3 ... xn yn
```

- `class_id`: 物件對應的類別 ID。
- `x1, y1, ... xn, yn`: 多邊形頂點座標，已正規化至 0～1 的範圍。

---

### **4. `data.yaml` 文件格式**

```yaml
train: /path/to/dataset/images/train
val: /path/to/dataset/images/val

nc: 6  # 類別數量
names:
  - black tongue coating
  - map tongue coating
  - purple tongue coating
  - red tongue yellow fur thick greasy fur
  - The red tongue is thick and greasy
  - The white tongue is thick and greasy
```

- `train`、`val`：指向訓練、驗證圖片的資料夾路徑。
- `nc`：總類別數量。
- `names`：類別名稱列表，順序需與標註文件中的 `class_id` 對應。

---

### **5. 標註數據轉換流程**

1. **讀取原始標註**：如 COCO、VOC 或自定格式等。  
2. **正規化多邊形點**：根據圖片寬高，將絕對座標轉換為 0～1 區間。  
3. **生成 `.txt` 標註檔**：依據 YOLO 的多邊形格式輸出。

---

### **6. 範例結構**

- **圖片 (e.g. `images/train/image1.jpg`)**  
  此為原始影像檔。

- **標註 (e.g. `labels/train/image1.txt`)**  
  ```plaintext
  0 0.1 0.2 0.2 0.2 0.2 0.3 0.1 0.3
  1 0.5 0.5 0.6 0.5 0.6 0.6 0.5 0.6
  ```
  - 第一行：類別 0 的四邊形。
  - 第二行：類別 1 的四邊形。

- **`data.yaml`**  
  ```yaml
  train: /dataset/images/train
  val: /dataset/images/val

  nc: 6
  names:
    - black tongue coating
    - map tongue coating
    - purple tongue coating
    - red tongue yellow fur thick greasy fur
    - The red tongue is thick and greasy
    - The white tongue is thick and greasy
  ```

---

### **7. 檢查數據完整性**

在訓練前，可先檢查圖片與標註的對應是否正確：

```python
import os

def check_data(data_dir):
    images_dir = os.path.join(data_dir, 'images')
    labels_dir = os.path.join(data_dir, 'labels')

    for subset in ['train', 'val']:
        images = set(os.listdir(os.path.join(images_dir, subset)))
        labels = set(os.listdir(os.path.join(labels_dir, subset)))

        # 去除副檔名來比對名稱
        images = {os.path.splitext(f)[0] for f in images}
        labels = {os.path.splitext(f)[0] for f in labels}

        missing_images = labels - images
        missing_labels = images - labels

        print(f"Subset: {subset}")
        print(f"  Missing images: {missing_images}")
        print(f"  Missing labels: {missing_labels}")

check_data('/path/to/dataset')
```

- 確認每張圖片都有對應的標註檔，並且檔名相符。

---
[最新的yolo模型](https://docs.ultralytics.com/zh/tasks/segment/) 

In [ ]:
import os
import shutil
import random
import json
import cv2
import yaml
import numpy as np

# 原始資料夾路徑
data_dir = "data\Tongue_coating_classification"  # 資料夾根目錄，包含各個舌頭類型的子資料夾
output_dir = os.path.join(data_dir, "yolo_data")
images_dir = os.path.join(output_dir, "images")
labels_dir = os.path.join(output_dir, "labels")  # 新增 labels 資料夾來保存 txt 標註文件

# 設定每個舌頭類型的標籤
class_labels = {
    "black tongue coating": 0,
    "map tongue coating": 0,
    "purple tongue coating": 0,
    "red tongue yellow fur thick greasy fur": 0,
    "The red tongue is thick and greasy": 0,
    "The white tongue is thick and greasy": 0
}

# 建立資料夾結構
os.makedirs(os.path.join(images_dir, "train"), exist_ok=True)
os.makedirs(os.path.join(images_dir, "val"), exist_ok=True)
os.makedirs(os.path.join(labels_dir, "train"), exist_ok=True)
os.makedirs(os.path.join(labels_dir, "val"), exist_ok=True)

def create_txt_from_polygon(points, img_width, img_height, class_id):
    """將多邊形的點坐標寫入 .txt 文件"""
    # 正規化多邊形坐標 (將其轉換為 [0, 1] 範圍內)
    normalized_points = []
    for point in points:
        x = point[0] / img_width
        y = point[1] / img_height
        normalized_points.append((x, y))
    
    # 將正規化後的點寫入格式
    return [class_id] + [f"{x} {y}" for x, y in normalized_points]

# 收集圖片和標註檔案
image_paths = []
for class_name, class_id in class_labels.items():
    class_dir = os.path.join(data_dir, class_name)
    if not os.path.exists(class_dir):
        print(f"警告：{class_dir} 資料夾不存在，跳過...")
        continue

    for filename in os.listdir(class_dir):
        if filename.endswith(".json"):
            json_path = os.path.join(class_dir, filename)
            with open(json_path, 'r') as f:
                data = json.load(f)
            
            # 確認對應的圖片檔案
            img_filename = data["imagePath"]
            img_path = os.path.join(class_dir, img_filename)
            if not os.path.exists(img_path):
                print(f"警告：找不到圖片 {img_path}，跳過...")
                continue
            
            # 讀取圖片大小
            img_height, img_width = cv2.imread(img_path).shape[:2]
            image_paths.append(img_path)
            
            # 建立 .txt 文件，將每個物體的多邊形填入其中
            txt_output_path = os.path.join(labels_dir, "train" if random.random() < 0.8 else "val", 
                                          os.path.splitext(img_filename)[0] + ".txt")
            
            with open(txt_output_path, 'w') as txt_file:
                for shape in data["shapes"]:
                    if shape["label"] == class_name:
                        points = shape["points"]
                        # 生成對應的多邊形資料並寫入 .txt 文件
                        txt_file.write(" ".join(map(str, create_txt_from_polygon(points, img_width, img_height, class_id))) + "\n")

# 分配至訓練集和驗證集
random.shuffle(image_paths)
train_ratio = 0.8
train_count = int(len(image_paths) * train_ratio)

for i, img_path in enumerate(image_paths):
    subset = "train" if i < train_count else "val"
    filename = os.path.basename(img_path)
    
    # 複製圖片到對應的資料夾
    shutil.copy(img_path, os.path.join(images_dir, subset, filename))

# 建立 data.yaml 文件
data_yaml = {
    "train": "C:/Users/B20770/Desktop/yolo_tongue_coating/Tongue_coating_classification/yolo_data/images/train"
    "val": "C:/Users/B20770/Desktop/yolo_tongue_coating/Tongue_coating_classification/yolo_data/images/val"


    "nc": 1  # Number of classes, single class
    "names": ['segment']  # Class name
    "task": "segment"  # Task type: segmentation
}

with open(os.path.join(output_dir, "data.yaml"), 'w') as f:
    yaml.dump(data_yaml, f)

print("語意分割格式資料已處理並保存至 yolo_data 資料夾中。")
